<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/labeled(ang%2Canx%2Csad%2Cexc%2Cneu)_without_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy


In [ ]:
import numpy as np

# Load .npz file
data = np.load('/content/DatasetCaricatoNoImage.npz')

# Check what arrays are stored inside
print(data.files)

# Access arrays by name
for array_name in data.files:
    print(f"{array_name}: {data[array_name]}")


['arr_0']
arr_0: [[[28.213411 27.665922 28.723648 ... 27.510525 27.10244  27.157482]
  [23.35981  23.371222 23.344477 ... 21.225304 21.23307  21.698439]
  [20.515251 20.415539 20.414434 ... 19.743534 19.71544  20.037113]
  [18.16684  18.067276 18.765232 ... 18.43621  18.192768 18.360853]
  [17.0527   16.986307 18.354567 ... 17.650728 17.19495  17.025713]]

 [[28.217297 27.666159 28.73337  ... 27.51639  27.107931 27.1646  ]
  [23.35571  23.368097 23.342169 ... 21.2249   21.232828 21.69778 ]
  [20.513042 20.41324  20.412043 ... 19.74365  19.71521  20.036781]
  [18.166636 18.06655  18.765318 ... 18.435701 18.191483 18.359507]
  [17.05315  16.986612 18.355356 ... 17.651464 17.1949   17.025066]]

 [[28.22005  27.6672   28.740765 ... 27.526598 27.113993 27.17042 ]
  [23.349567 23.363235 23.338316 ... 21.2249   21.233492 21.698042]
  [20.509924 20.41043  20.409527 ... 19.742777 19.71422  20.035297]
  [18.165844 18.065481 18.765062 ... 18.434877 18.190392 18.358133]
  [17.053148 16.98615  18.3

In [ ]:
import numpy as np

# Load .npz file
data = np.load('/content/LabelsNoImage.npz')

# Check what arrays are stored inside
print(data.files)

# Access arrays by name
for array_name in data.files:
    print(f"{array_name}: {data[array_name]}")


['arr_0']
arr_0: [2 2 2 ... 0 0 0]


In [ ]:
import numpy as np

# Load .npz file
data = np.load('/content/SubjectsNoImage.npz')

# Check what arrays are stored inside
print(data.files)

# Access arrays by name
for array_name in data.files:
    print(f"{array_name}: {data[array_name]}")


['arr_0']
arr_0: [ 0  0  0 ... 14 14 14]


In [ ]:
import numpy as np
import pandas as pd

# Load the .npz file
data = np.load('DatasetCaricatoNoImage.npz')

# Print available keys to identify the correct one
print(data.files)

# Access the array using the correct key (replace 'arr_0' with the actual key if different)
eeg_data = data['arr_0']  # Assuming 'arr_0' is the correct key based on previous outputs

# Check the shape (should be 50910, 5, 62)
print(f"Original Shape: {eeg_data.shape}")

# Flatten each window (5x62) into a single row with 310 columns
flattened_data = eeg_data.reshape(50910, -1)

# Convert to DataFrame
df = pd.DataFrame(flattened_data)

# Save to CSV
df.to_csv("eeg_data_flattened.csv", index=False)

print("✅ Flattened data saved to 'eeg_data_flattened.csv'")

['arr_0']
Original Shape: (50910, 5, 62)
✅ Flattened data saved to 'eeg_data_flattened.csv'


In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.1 MB/s eta 0:00:00


In [ ]:
!pip install PyWavelets

import numpy as np
import pandas as pd
import scipy.signal as signal
import pywt
from scipy import stats

# Feature Extraction Functions
def extract_time_features(eeg_signal):
    return [
        np.mean(eeg_signal),
        np.std(eeg_signal),
        stats.kurtosis(eeg_signal),
        stats.skew(eeg_signal)
    ]

def extract_frequency_features(eeg_signal, sampling_rate=256):
    freqs, psd = signal.welch(eeg_signal, fs=sampling_rate, nperseg=min(256, len(eeg_signal)))
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 14), 'beta': (14, 30), 'gamma': (30, 40)}
    return [np.sum(psd[(freqs >= low) & (freqs < high)]) for low, high in bands.values()]

def extract_wavelet_features(eeg_signal, wavelet='db4'):
    max_level = pywt.dwt_max_level(len(eeg_signal), pywt.Wavelet(wavelet))
    level = min(3, max_level)
    coeffs = pywt.wavedec(eeg_signal, wavelet, level=level)
    return [np.mean(np.abs(c)) for c in coeffs]

# Process CSV and Combine Features & Labels
def process_csv_and_add_labels(csv_file, label_file, output_csv):
    df = pd.read_csv(csv_file)

    # Access the array within the npz file using a key, e.g., 'arr_0'
    labels_data = np.load(label_file)
    # Assuming 'arr_0' is the key for your labels array (adjust if needed)
    labels = labels_data['arr_0'].flatten()

    all_features = []
    for index, row in df.iterrows():
        eeg_signal = row.values
        time_features = extract_time_features(eeg_signal)
        frequency_features = extract_frequency_features(eeg_signal)
        wavelet_features = extract_wavelet_features(eeg_signal)
        all_features.append(time_features + frequency_features + wavelet_features)

    feature_cols = (
        ['mean', 'std', 'kurtosis', 'skewness'] +
        ['delta_power', 'theta_power', 'alpha_power', 'beta_power', 'gamma_power'] +
        [f'wavelet_{i}' for i in range(len(all_features[0]) - 9)]  # dynamic wavelet columns
    )

    feature_df = pd.DataFrame(all_features, columns=feature_cols)
    feature_df['label'] = labels
    feature_df.to_csv(output_csv, index=False)

    print(f"✅ Features with labels saved to '{output_csv}'")

# Example usage
process_csv_and_add_labels('eeg_data_flattened.csv', '/content/LabelsNoImage.npz', 'eeg_features_with_labels.csv')

✅ Features with labels saved to 'eeg_features_with_labels.csv'


In [ ]:
import pandas as pd

# Load extracted features
features_df = pd.read_csv("/content/eeg_features_with_labels.csv")

def assign_emotion_label(row):
    delta, theta, alpha, beta, gamma = row['delta_power'], row['theta_power'], row['alpha_power'], row['beta_power'], row.get('gamma_power', 0)

    if beta > alpha and beta > gamma and beta > theta and beta > delta:
        return 'Excited'
    elif gamma > beta and gamma > alpha and gamma > theta and gamma > delta:
        return 'Angry'
    elif alpha > theta and alpha > beta and alpha > delta and alpha > gamma:
        return 'Calm'
    elif theta > alpha and theta > beta and theta > gamma and theta > delta:
        return 'Anxious'
    elif delta > alpha and delta > theta and delta > beta and delta > gamma:
        return 'Sad'
    elif beta > alpha and gamma > theta and gamma > delta:
        return 'Happy'
    else:
        return 'Neutral'


# Apply labeling function
features_df['label'] = features_df.apply(assign_emotion_label, axis=1)

# Save labeled dataset
features_df.to_csv("labeled_data.csv", index=False)
print("✅ Labeled dataset saved as labeled_eeg_data.csv")

✅ Labeled dataset saved as labeled_data_with_neutral.csv


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install --upgrade --force-reinstall tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached setuptools-75.8.2-py3-none-any.whl.metadata (6.7 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached termcolor-2.5.0-py3-none-

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# 1. Load Data
df = pd.read_csv('/content/labeled_data.csv')  # Make sure it has features + "label" column

# 2. Split into Features & Labels
X = df.drop(columns=['label']).values  # All columns except 'label'
y = df['label'].values  # Emotion labels

# 3. Encode Labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

# One-hot encode the labels (required for multi-class classification)
y_onehot = to_categorical(integer_encoded)

# 4. Reshape for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# 5. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# 6. Build LSTM Model
model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
model.add(Dense(y_onehot.shape[1], activation='softmax'))  # Number of classes = y_onehot.shape[1]
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 8. Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

# 9. Save Model
model.save('eeg_emotion_lstm_model.h5')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1273/1273 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8597 - loss: 0.4177 - val_accuracy: 0.9399 - val_loss: 0.1536
Epoch 2/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9520 - loss: 0.1380 - val_accuracy: 0.9626 - val_loss: 0.1023
Epoch 3/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9621 - loss: 0.1099 - val_accuracy: 0.9600 - val_loss: 0.1021
Epoch 4/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9638 - loss: 0.0982 - val_accuracy: 0.9725 - val_loss: 0.0762
Epoch 5/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9678 - loss: 0.0876 - val_accuracy: 0.9770 - val_loss: 0.0686
Epoch 6/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9689 - loss: 0.0824 - val_accuracy: 0.9677 - val_loss: 0.0808
Epoch 7/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9679 - loss: 0.0813 - val_accuracy: 0.9785 - val_loss: 0.0598
Epoch 8/50
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9709 - loss: 0.0737 - val_accura

Test Accuracy: 0.96


In [ ]:
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 96.48%


In [ ]:
model.save('eeg_emotion_lstm_model.h5')


In [ ]:
# ... (your existing code) ...

# 9. Save Model
model.save('eeg_emotion_lstm_model.keras')  # Changed from .h5 to .keras

# ... (rest of your code) ...

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/labeled_data.csv")

# Print the shape of the entire dataset
print(f"Full dataset shape: {df.shape}")

# Features (first 13 columns) and label (last column)
X = df.iloc[:, :-1].values   # 13 features
y = df.iloc[:, -1].values    # 1 label (emotion)

# Print individual shapes
print(f"Features shape (X): {X.shape}")
print(f"Labels shape (y): {y.shape}")


Full dataset shape: (50910, 14)
Features shape (X): (50910, 13)
Labels shape (y): (50910,)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
df = pd.read_csv("/content/labeled_data.csv")

# Features and labels
X = df.iloc[:, :-1].values   # 13 features
y = df.iloc[:, -1].values    # Labels (e.g., Happy, Sad, etc.)

# Encode labels to numbers (0, 1, 2...)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features (recommended for GANs and LSTM)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape to (samples, time_steps, features) for LSTM (time_steps = 1 for static data)
X_reshaped = X_scaled.reshape(-1, 1, 13)

print(f"Features shape: {X_reshaped.shape}, Labels shape: {y_encoded.shape}")


Features shape: (50910, 1, 13), Labels shape: (50910,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, LeakyReLU

# Generator
def build_generator(latent_dim):
    model = Sequential([
        Dense(64, activation="relu", input_dim=latent_dim),
        Dense(13, activation="tanh"),  # 13 features
        Reshape((1, 13))  # 1 time step, 13 features
    ])
    return model

# Discriminator
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=(1, 13)),
        Dense(64, activation=LeakyReLU(0.2)),
        Dense(1, activation="sigmoid")  # Real or Fake
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [ ]:
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# Combined GAN
discriminator.trainable = False  # Freeze discriminator when training generator
gan = Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="adam")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
epochs = 2000
batch_size = 32

for epoch in range(epochs):
    # Real samples (from your dataset)
    idx = np.random.randint(0, X_reshaped.shape[0], batch_size)
    real_samples = X_reshaped[idx]

    # Fake samples (generated by the GAN)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_samples = generator.predict(noise)

    # Train discriminator (real = 1, fake = 0)
    d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator (want discriminator to classify fakes as real = 1)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]:.4f}, G Loss: {g_loss:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, D Loss: 0.6462, G Loss: 0.7039
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━